# Scrap Page Details to JSON File

In [ ]:
import requests
import json

# RAPID API credentials
url = "https://facebook-pages-scraper2.p.rapidapi.com/get_facebook_pages_details"
headers = {
    "x-rapidapi-host": "facebook-pages-scraper2.p.rapidapi.com",
    "x-rapidapi-key": "YOUR_API_KEY_HERE" # Insert API Key
}

# Target Facebook Page link
params = {"link": "https://www.facebook.com/YourPageName"}  # Insert The Facebook Page 

# API Request
response = requests.get(url, headers=headers, params=params)

# Save JSON Response to a File
if response.status_code == 200:
    data = response.json()  # Full JSON response
    
    # Save to a file
    with open('facebook_page_details.json', 'w') as f:
        json.dump(data, f, indent=4)
    print("JSON saved successfully to 'facebook_page_details.json'")
else:
    print("Failed to retrieve data:", response.status_code, response.text)


# Convert JSON files to CSV

In [ ]:
from datetime import datetime, timedelta

# Function to convert UTC time to WIB (UTC+7)
def convert_to_wib(utc_time_str):
    try:
        utc_time = datetime.strptime(utc_time_str, '%Y-%m-%dT%H:%M:%S%z')
        wib_time = utc_time + timedelta(hours=7)  # Add 7 hours for WIB
        return wib_time.strftime('%Y-%m-%d %H:%M:%S')  # Format to readable string
    except Exception:
        return utc_time_str  # Return original if not a valid datetime

# Step 1: Read the JSON file
json_file = 'facebook_page_details.json'
with open(json_file, 'r') as f:
    data = json.load(f)

# Step 2: Normalize JSON data into a DataFrame
df = pd.json_normalize(data)

# Step 3: Convert date-time columns to WIB
# Replace with actual column names that may contain time-related data
date_time_columns = ['creation_date', 'created_time']  # Adjust if needed
for col in date_time_columns:
    if col in df.columns:
        df[col] = df[col].apply(convert_to_wib)

# Step 4: Save the DataFrame to CSV
csv_file = 'facebook_page_details_wib.csv'
df.to_csv(csv_file, index=False)
print(f"Data successfully converted and saved to '{csv_file}'")

# Scrap Post Details to JSON File

In [ ]:
import requests
import json

# API Endpoint and Headers
url = "https://facebook-pages-scraper2.p.rapidapi.com/get_facebook_posts_details" 
headers = {
    "x-rapidapi-host": "facebook-pages-scraper2.p.rapidapi.com",
    "x-rapidapi-key": "YOUR_API_KEY_HERE" # Insert API Key
}

# API Parameters (Target Facebook Page)
params = {"link": "https://www.facebook.com/YourPageName", "timezone": "UTC"}  # Insert The Facebook Page 

# Make the API Request
response = requests.get(url, headers=headers, params=params)

# Save JSON to File
if response.status_code == 200:
    data = response.json()  # Full JSON response
    
    # Save JSON data to a file
    json_filename = "facebook_posts.json"
    with open(json_filename, "w") as f:
        json.dump(data, f, indent=4)  # Pretty print JSON for readability
    print(f"JSON data successfully saved to '{json_filename}'")
else:
    print("Failed to retrieve data:", response.status_code, response.text)


# Convert JSON files to CSV

In [ ]:
import json
import pandas as pd
from datetime import datetime, timedelta

# Function to convert UTC time to WIB (UTC+7)
def convert_to_wib(utc_time_str):
    try:
        utc_time = datetime.strptime(utc_time_str, '%Y-%m-%dT%H:%M:%S%z')
        wib_time = utc_time + timedelta(hours=7)  # Add 7 hours for WIB
        return wib_time.strftime('%Y-%m-%d %H:%M:%S')  # Format to readable string
    except Exception:
        return utc_time_str  # Return original if not a valid datetime

# Step 1: Load JSON Data
json_filename = "facebook_posts.json"
with open(json_filename, "r") as f:
    data = json.load(f)

# Step 2: Extract Posts Data
posts = data.get('data', {}).get('posts', [])
if posts:
    post_data = []
    for post in posts:
        details = post.get('details', {})
        reactions = post.get('reactions', {})
        values = post.get('values', {})
        
        # Flatten post data
        post_data.append({
            "post_id": details.get("post_id"),
            "post_link": details.get("post_link"),
            "text": values.get("text"),
            "share_count": details.get("share_count"),
            "comments_count": details.get("comments_count"),
            "total_reaction_count": reactions.get("total_reaction_count"),
            "like_count": reactions.get("Like", 0),
            "haha_count": reactions.get("Haha", 0),
            "love_count": reactions.get("Love", 0),
            "care_count": reactions.get("Care", 0),
            "publish_time": convert_to_wib(values.get("publish_time", "")),
            "media_type": values.get("is_media"),
            "media_url": values.get("photo_image", {}).get("uri", "")
        })
    
    # Step 3: Convert to DataFrame and Save as CSV
    df = pd.DataFrame(post_data)
    csv_filename = "facebook_posts_wib.csv"
    df.to_csv(csv_filename, index=False)
    print(f"Post data successfully converted and saved to '{csv_filename}'")
else:
    print("No posts found in the JSON file.")
